In [2]:
# training dataset
from datasets import load_dataset, load_metric, list_metrics
import os
import sys 
# add transformer into path
sys.path.insert(0, '/disk3/Haonan/yanbo_random/bert_finetune/transformers/src')
import transformers
print(transformers.__file__)

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np
import torch
from torch.optim.lr_scheduler import StepLR

from pkg_resources import packaging

import torch.optim as optim
from peft import get_peft_model, prepare_model_for_int8_training

from torch.distributed.fsdp.fully_sharded_data_parallel import CPUOffload

from pathlib import Path
from train_utils import train, evaluation, setup
from training_config import train_config as TRAIN_CONFIG

# import self_def_datasets.dataset as sds

load_ckpt_path = "/disk3/Haonan/yanbo_random/bert_finetune/checkpoint/train_basic_yelp" # gelu
ckpt_path = "/disk3/Haonan/yanbo_random/bert_finetune/checkpoint/train_basic_yelp_relu_1000_sparsity_0"

log_path = "/disk3/Haonan/yanbo_random/bert_finetune/logs/train_basic_yelp"
train_config = TRAIN_CONFIG()
if train_config.enable_fsdp:
    setup()
    # torchrun specific
    local_rank = int(os.environ["LOCAL_RANK"])
    rank = int(os.environ["RANK"])
    world_size = int(os.environ["WORLD_SIZE"])



/home/dltp_haonan/anaconda3/envs/llamaRelu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/disk3/Haonan/yanbo_random/bert_finetune/transformers/src/transformers/__init__.py


In [3]:
# torch.multiprocessing.set_sharing_strategy('file_system')

dataset = load_dataset("yelp_review_full")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)
# 这个样例数据集不大，300M
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))


In [19]:
# small_train_dataset = small_train_dataset['input_ids','label', 'token_type_ids', 'attention_mask']
# small_eval_dataset = small_eval_dataset['input_ids','label', 'token_type_ids', 'attention_mask']
from torch.utils.data import Dataset
import torch
class SelfDefDataset(Dataset):
    def __init__(self, dataset_dict):
        self.dataset_dict = dataset_dict

    def __len__(self):
        return len(self.dataset_dict)

    def __getitem__(self, idx):
        # print('idx : ',idx)
        # item = {key: torch.tensor(val[idx]) for key, val in self.dataset_dict.features.items()}
        input = torch.tensor(self.dataset_dict['input_ids'][idx])
        mask = torch.tensor(self.dataset_dict['attention_mask'][idx])
        label = torch.tensor(self.dataset_dict['label'][idx])
        item = {'input_ids': input, 'attention_mask': mask, 'labels': label}
        return item



In [20]:
train_data = SelfDefDataset(small_train_dataset)
eval_data = SelfDefDataset(small_eval_dataset)

In [21]:
# small_eval_dataset['input_ids','label']
# small_eval_dataset['attention_mask'][0]
print(small_train_dataset)
# small_eval_dataset.features.items()
type(small_eval_dataset)
# small_train_dataset['label'][0]

# def __getitem__(self, idx):
#     item = {key: torch.tensor(val[idx]) for key, val in self.dataset_dict.items()}
#     return item


Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 100
})


datasets.arrow_dataset.Dataset

In [22]:
print('train data : ',train_data[0])

train data :  {'input_ids': tensor([  101,   146, 27438,  1142,  4202,   119,   146,   112,  1396,  1151,
         1106,  3924,  8412,  1187,   146,  9981,  1106,  1129,   170, 13395,
         7589,  2288,  1107,  1413,   117,  6322,  8796,  5030,  7424,   117,
         1105,  1104,  1736,  1103,  9230,   112,   188,  2319,   119,  1109,
        20400,  1132,  1177,  1177,  7284, 10455,   119,  1109,  3172,  1110,
         7688,  4931,  1105,  1119,  2228,  1296,  7329,  1118,  1289,  1114,
         1126, 10965,  2971,  1104,  8188,   119,  1192, 13224,  3940,  1303,
         3713,   106,   106,   106,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0, 

In [23]:
print('text : ',small_eval_dataset['text'][0])
print('label : ',small_eval_dataset['label'][0])
print('train data : ',train_data[0])
train_dataloader = torch.utils.data.DataLoader(
    train_data,
    num_workers=train_config.num_workers_dataloader,
    batch_size=8,
    # pin_memory=True,
    # **train_dl_kwargs,
)

if train_config.run_validation:
    eval_dataloader = torch.utils.data.DataLoader(
        eval_data,
        num_workers=train_config.num_workers_dataloader,
        # pin_memory=True,
        batch_size=8,
        # **val_dl_kwargs,
    )

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')



text :  Kabuto is your run-of-the-mill Japanese Steakhouse. Different stations with chefs slinging shrimp tails around the communal dining areas like it's a lunchtime magic show. Always a plethora of laughs and gags going around the group. \n\nThis place is great for lunch. $9 and 30 minutes and you're out the door. Uhhh...If I'm craving a salad with ginger dressing, which I always am, (you do too. admit it) fried rice, steak, shrimp and white sauce (DUDE) then Kabuto is king of lunch options in my book. Always super clean and full of kindhearted staff. The parking lot is super difficult to get in and out of though. 51 traffic at lunch is a beast. Good luck getting stuck behind someone trying to cut across traffic at 12pm on a weekday. It's murder. This place would greatly benefit from another exit/entrance or a stoplight. Here's hoping....\n\nYou can't really shake a stick at balanced lunch when you can have soup or a salad, veggies, fried rice and a choice of a protein for under $10 

In [24]:

# use ft model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5) # gelu
# model = AutoModelForSequenceClassification.from_pretrained(f"{load_ckpt_path}/checkpoint-18750", num_labels=5)
# metric=load_metric("accuracy")
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
model.to(device)
# print(next(model.parameters()).device)

optimizer = optim.AdamW(
    model.parameters(),
    lr=train_config.lr,
    weight_decay=train_config.weight_decay,
)
scheduler = StepLR(optimizer, step_size=1, gamma=train_config.gamma)
# Start the training process
results = train(
    model,
    train_dataloader,
    eval_dataloader,
    tokenizer,
    optimizer,
    scheduler,
    train_config.gradient_accumulation_steps,
    train_config
)
[print(f'Key: {k}, Value: {v}') for k, v in results.items()]



BertIntermediate step idx = 0
BertIntermediate step idx = 0
BertIntermediate step idx = 0
BertIntermediate step idx = 0
BertIntermediate step idx = 0
BertIntermediate step idx = 0
BertIntermediate step idx = 0
BertIntermediate step idx = 0
BertIntermediate step idx = 0
BertIntermediate step idx = 0
BertIntermediate step idx = 0
BertIntermediate step idx = 0


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


start forward 0
BertIntermediate step idx + 1
end forward 1
start forward 0
BertIntermediate step idx + 1
end forward 1
start forward 0
BertIntermediate step idx + 1
end forward 1
start forward 0
BertIntermediate step idx + 1
end forward 1
start forward 0
BertIntermediate step idx + 1
end forward 1
start forward 0
BertIntermediate step idx + 1
end forward 1
start forward 0
BertIntermediate step idx + 1
end forward 1
start forward 0
BertIntermediate step idx + 1
end forward 1
start forward 0
BertIntermediate step idx + 1
end forward 1
start forward 0
BertIntermediate step idx + 1
end forward 1
start forward 0
BertIntermediate step idx + 1
end forward 1
start forward 0
BertIntermediate step idx + 1
end forward 1


start forward 1
BertIntermediate step idx + 1
end forward 2
start forward 1
BertIntermediate step idx + 1
end forward 2
start forward 1
BertIntermediate step idx + 1
end forward 2
start forward 1
BertIntermediate step idx + 1
end forward 2
start forward 1
BertIntermediate step idx + 1
end forward 2
start forward 1
BertIntermediate step idx + 1
end forward 2
start forward 1
BertIntermediate step idx + 1
end forward 2
start forward 1
BertIntermediate step idx + 1
end forward 2
start forward 1
BertIntermediate step idx + 1
end forward 2
start forward 1
BertIntermediate step idx + 1
end forward 2
start forward 1
BertIntermediate step idx + 1
end forward 2
start forward 1
BertIntermediate step idx + 1
end forward 2


start forward 2
BertIntermediate step idx + 1
end forward 3
start forward 2
BertIntermediate step idx + 1
end forward 3
start forward 2
BertIntermediate step idx + 1
end forward 3
start forward 2
BertIntermediate step idx + 1
end forward 3
start forward 2
BertIntermediate step idx + 1
end forward 3
start forward 2
BertIntermediate step idx + 1
end forward 3
start forward 2
BertIntermediate step idx + 1
end forward 3
start forward 2
BertIntermediate step idx + 1
end forward 3
start forward 2
BertIntermediate step idx + 1
end forward 3
start forward 2
BertIntermediate step idx + 1
end forward 3
start forward 2
BertIntermediate step idx + 1
end forward 3
start forward 2
BertIntermediate step idx + 1
end forward 3


...

start forward 23
BertIntermediate step idx + 1
end forward 24
start forward 23
BertIntermediate step idx + 1
end forward 24
start forward 23
BertIntermediate step idx + 1
end forward 24
start forward 23
BertIntermediate step idx + 1
end forward 24
start forward 23
BertIntermediate step idx + 1
end forward 24
start forward 23
BertIntermediate step idx + 1
end forward 24
start forward 23
BertIntermediate step idx + 1
end forward 24
start forward 23
BertIntermediate step idx + 1
end forward 24
start forward 23
BertIntermediate step idx + 1
end forward 24
start forward 23
BertIntermediate step idx + 1
end forward 24
start forward 23
BertIntermediate step idx + 1
end forward 24
start forward 23
BertIntermediate step idx + 1
end forward 24


start forward 24
BertIntermediate step idx + 1
end forward 25
start forward 24
BertIntermediate step idx + 1
end forward 25
start forward 24
BertIntermediate step idx + 1
end forward 25
start forward 24
BertIntermediate step idx + 1
end forward 25
start forward 24
BertIntermediate step idx + 1
end forward 25
start forward 24
BertIntermediate step idx + 1
end forward 25
start forward 24
BertIntermediate step idx + 1
end forward 25
start forward 24
BertIntermediate step idx + 1
end forward 25
start forward 24
BertIntermediate step idx + 1
end forward 25
start forward 24
BertIntermediate step idx + 1
end forward 25
start forward 24
BertIntermediate step idx + 1
end forward 25
start forward 24
BertIntermediate step idx + 1
end forward 25
start forward 25
BertIntermediate step idx + 1
end forward 26
start forward 25
BertIntermediate step idx + 1
end forward 26
start forward 25


evaluating Epoch: 100%|██████████| 13/13 [00:05<00:00,  2.45it/s]


BertIntermediate step idx + 1
end forward 26
start forward 25
BertIntermediate step idx + 1
end forward 26
start forward 25
BertIntermediate step idx + 1
end forward 26
start forward 25
BertIntermediate step idx + 1
end forward 26
start forward 25
BertIntermediate step idx + 1
end forward 26
start forward 25
BertIntermediate step idx + 1
end forward 26
start forward 25
BertIntermediate step idx + 1
end forward 26
start forward 25
BertIntermediate step idx + 1
end forward 26
start forward 25
BertIntermediate step idx + 1
end forward 26
start forward 25
BertIntermediate step idx + 1
end forward 26
 eval_ppl=tensor(4.9154, device='cuda:0') eval_epoch_loss=tensor(1.5924, device='cuda:0')


AttributeError: 'NoneType' object has no attribute 'checkpoint_type'